# **Time Series** 🕜

Decripcion Aqui...

**Authors:**
- [Adrian Flores](https://github.com/adrianRFlores)
- [Andrea Ramirez](https://github.com/Andrea-gt)

-----

## **Import Libraries** ⬇️

In [32]:
 #!pip install dataprep

In [33]:
# Data manipulation and visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Standard libraries
import warnings
warnings.filterwarnings('ignore')

from dataprep.datasets import load_dataset
from dataprep.eda import create_report, plot, plot_correlation, plot_missing

# ===== ===== Reproducibility Seed ===== =====
# Set a fixed seed for the random number generator for reproducibility
random_state = 42

# Set matplotlib inline
%matplotlib inline

# Set default figure size
plt.rcParams['figure.figsize'] = (6, 4)

# Define custom color palette
palette = sns.color_palette("viridis", 12)

## **Data Upload** 📄

In [34]:
def read_and_process_excel(file_names):
    dfs = []  # Initialize an empty list to store DataFrames

    for file_name in file_names:
        # Read the Excel file while skipping the first six rows of headers
        df = pd.read_excel(file_name, skiprows=6)

        # Drop the last three rows from the DataFrame to remove any unwanted data
        df = df.iloc[:-3]

        # Convert the 'Fecha' column to datetime format
        df['Fecha'] = pd.to_datetime(df['Fecha'])

        # Set the 'Fecha' column as the index of the DataFrame
        df.set_index('Fecha', inplace=True)

        # Select only the specified columns and create a new column 'Diesel' by combining 'Diesel alto azufre' and 'Diesel bajo azufre'
        df['Diesel'] = df['Diesel alto azufre'].fillna(0) + df['Diesel bajo azufre'].fillna(0)

        # Select only the relevant columns: Gasolina regular, Gasolina superior, Gas licuado de petróleo, and the new Diesel column
        df = df[['Gasolina regular', 'Gasolina superior', 'Gas licuado de petróleo', 'Diesel']]

        # Append the processed DataFrame to the list
        dfs.append(df)

    return dfs  # Return the list of DataFrames

In [35]:
# List of Excel file names to be processed
file_names = ["consumo.xlsx", "importacion.xlsx"]
dataset_names = ["Consumo", "Importacion"]

# Call the function to read and process the Excel files, storing the result in the dataframes variable
dataframes = read_and_process_excel(file_names)

## **Exploratory Analysis** 🔎

### **(1) Descripción y Análisis de los Datos**

In [36]:
for i, df in enumerate(dataframes):
  # Get the number of rows in the merged DataFrame
  rows_num = df.shape[0]

  # Print the number of records in the DataFrame
  print(f"The given dataset {dataset_names[i]} has", rows_num, "registers.")

The given dataset Consumo has 293 registers.
The given dataset Importacion has 281 registers.


**Observaciones 💡 -->**

> - El primer conjunto de datos se centra en la recopilación de información histórica sobre el consumo nacional de petróleo y productos petroleros en Guatemala. Este conjunto abarca entradas desde enero de 2000 hasta la fecha actual. La información fue obtenida de la página oficial del [Ministerio de Energía y Minas](https://mem.gob.gt/que-hacemos/hidrocarburos/comercializacion-downstream/estadisticas-de-mercado/#1599459297938-cd7460d6-8212). Cuenta con alrededor de 293 registros y 4 columnas.

> - El segundo conjunto de datos se centra en la recopilación de información histórica sobre la importación de productos derivados del petróleo en Guatemala. Este conjunto abarca entradas desde enero de 2001 hasta la fecha actual. La información fue obtenida de la página oficial del [Ministerio de Energía y Minas](https://mem.gob.gt/que-hacemos/hidrocarburos/comercializacion-downstream/estadisticas-de-mercado/#1599459009048-ab105b3e-20a3). Cuenta con alrededor de 281 registros y 4 columnas, de manera similar al conjunto anterior.

In [37]:
for df in dataframes:
  # Basic information about the dataset
  df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 293 entries, 2000-01-01 to 2024-05-01
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gasolina regular         293 non-null    float64
 1   Gasolina superior        293 non-null    float64
 2   Gas licuado de petróleo  293 non-null    float64
 3   Diesel                   293 non-null    float64
dtypes: float64(4)
memory usage: 11.4 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 281 entries, 2001-01-01 to 2024-05-01
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gasolina regular         281 non-null    float64
 1   Gasolina superior        281 non-null    float64
 2   Gas licuado de petróleo  281 non-null    float64
 3   Diesel                   281 non-null    float64
dtypes: float64(4)
memory usage: 11.0 KB


**Observaciones 💡 -->**

> - En este análisis observamos que no es necesario realizar alteraciones en los tipos de las variables en ninguno de los conjuntos de datos, ya que todas están definidas de manera adecuada. Es relevante destacar que, aunque no se observan valores nulos en el conjunto de datos, existe la posibilidad de que algunos de ellos estén codificados de manera diferente. Por lo tanto, es crucial tener esto en cuenta durante el proceso de limpieza. Sin embargo, a medida que se avance en el análisis exploratorio, se podrá obtener información más detallada al respecto.

### **(2) Clasificación de las Variables**

**Conjunto de Datos de Consumo -->**

| Nombre                       | Descripción                                                          | Clasificación  |
|------------------------------|----------------------------------------------------------------------|----------------|
| Fecha                        | Fecha de registro de los datos de consumo de combustibles.            | Categórica     |
| Gasolina regular             | Consumo nacional de gasolina regular en barriles de 42 galones.       | Numérica       |
| Gasolina superior            | Consumo nacional de gasolina superior en barriles de 42 galones.      | Numérica       |
| Gas licuado de petróleo      | Consumo nacional de gas licuado de petróleo en barriles de 42 galones.| Numérica       |
| Diesel                       | Consumo nacional de diesel en barriles de 42 galones.                 | Numérica       |


**Conjunto de Datos de Importación -->**

| Nombre                       | Descripción                                                               | Clasificación  |
|------------------------------|---------------------------------------------------------------------------|----------------|
| Fecha                        | Fecha de registro de la importación de productos derivados del petróleo.  | Categórica     |
| Gasolina regular             | Importación nacional de gasolina regular en barriles de 42 galones.      | Numérica       |
| Gasolina superior            | Importación nacional de gasolina superior en barriles de 42 galones.     | Numérica       |
| Gas licuado de petróleo      | Imporatación nacional de gas licuado de petróleo en barriles de 42 galones.| Numérica       |
| Diesel                       | Importación nacional de diesel en barriles de 42 galones.                | Numérica       |

### **(2) Exploración y Limpieza Inicial de los Datos**

#### **🖊 Modificación de Etiquetas de Variables -->**

Para facilitar la comprensión y el manejo del conjunto de datos, se procederá a modificar los nombres de las variables. Este cambio permitirá una organización más clara y una interpretación más precisa de la información.

In [38]:
# Dictionary to rename columns for better readability
rename_col = {
    'Gasolina regular': 'gasoline_regular',    # Renaming 'Gasolina regular' to 'gasoline_regular'
    'Gasolina superior': 'gasoline_superior',  # Renaming 'Gasolina superior' to 'gasoline_superior'
    'Gas licuado de petróleo': 'liquefied_gas', # Renaming 'Gas licuado de petróleo' to 'liquefied_gas'
    'Diesel': 'diesel'                        # Renaming 'Diesel' to 'diesel'
}

In [44]:
for i, df in enumerate(dataframes):
  # Use a pandas function to rename the current function
  df = df.rename(columns = rename_col)
  # Change the index name from 'Fecha' to 'date'
  df.rename_axis('date', inplace=True)
  # Save changes
  dataframes[i] = df
  print(df.head(2))

            gasoline_regular  gasoline_superior  liquefied_gas     diesel
date                                                                     
2000-01-01         202645.20          308156.82  194410.476190  634667.06
2000-02-01         205530.96          307766.31  174710.552381  642380.66
            gasoline_regular  gasoline_superior  liquefied_gas     diesel
date                                                                     
2001-01-01         177776.50          373963.96  194065.738095  566101.99
2001-02-01         123115.99          243091.07  170703.380952  489525.80


#### **🥇 Top 10 Valores Más Frecuentes para Cada Columna (Tablas de Frecuencia) -->**

In [45]:
def get_top_ten(df):
  # See what are the 10 most frequent values for each of the dataframe columns
  for column in df.columns:
      frequency_values = df[column].value_counts().head(10)
      print("Top 10 most frequent values for column '{}':".format(column))
      for index, (value, frequency) in enumerate(frequency_values.items(), start=1):
          print("{:<5} {:<30} {:<10}".format(index, value, frequency))
      print("\n===============================================")

**Conjunto de Datos de Consumo -->**

In [46]:
get_top_ten(dataframes[0])

Top 10 most frequent values for column 'gasoline_regular':
1     202645.2                       1         
2     415570.94999999995             1         
3     456946.75                      1         
4     485995.83                      1         
5     465270.48000000004             1         
6     450758.18000000005             1         
7     468710.37999999995             1         
8     467472.2                       1         
9     486255.94000000006             1         
10    450916.35                      1         

Top 10 most frequent values for column 'gasoline_superior':
1     308156.82                      1         
2     483594.8300000001              1         
3     548491.65                      1         
4     585748.5                       1         
5     562778.84                      1         
6     537583.25                      1         
7     551000.79                      1         
8     545605.47                      1         
9     575720.92 

**Conjunto de Datos de Importación -->**

In [47]:
get_top_ten(dataframes[1])

Top 10 most frequent values for column 'gasoline_regular':
1     177776.5                       1         
2     397477.4                       1         
3     473659.59                      1         
4     476784.16000000003             1         
5     455850.75999999995             1         
6     567786.57                      1         
7     512202.57                      1         
8     526652.73                      1         
9     528347.81                      1         
10    469140.06                      1         

Top 10 most frequent values for column 'gasoline_superior':
1     373963.96                      1         
2     502082.45                      1         
3     526573.96                      1         
4     725548.16                      1         
5     546154.65                      1         
6     677892.97                      1         
7     625218.9                       1         
8     618366.1                       1         
9     615900.31 

**Observaciones 💡 -->**

> - En las tablas de frecuencia de los conjuntos de datos, observamos que los valores son únicos. Por lo tanto, procederemos a realizar un análisis descriptivo de las estadísticas de las columnas utilizando la función `describe()`.

#### **🔢 Estadísticas de Conjunto de Datos -->**

**Conjunto de Datos de Consumo -->**

In [48]:
dataframes[0].describe()

,gasoline_regular,gasoline_superior,liquefied_gas,diesel
count,293.000000,293.000000,293.000000,2.930000e+02
mean,386490.157486,464562.247013,312350.791082,8.697828e+05
std,209541.304568,108349.195858,109315.234878,2.185603e+05
min,160741.900000,300242.781667,167818.137897,5.076627e+05
25%,214509.650000,382078.880000,225886.426190,6.996108e+05
50%,285089.040000,420238.710000,262255.138095,8.076921e+05
75%,524242.310000,565665.720000,393901.921429,1.032813e+06
max,894533.140000,768102.830000,588892.160000,1.463008e+06


**Observaciones 💡 -->**

El análisis descriptivo del consumo nacional de petróleo y productos petroleros en Guatemala revela variaciones significativas entre los diferentes tipos de combustibles.

> - El consumo promedio de gasolina regular es de aproximadamente 386,490 unidades, con una desviación estándar considerable que indica alta variabilidad. La gasolina superior, con un consumo medio de 464,562 unidades, presenta una desviación estándar menor, sugiriendo una distribución más estable. El gas licuado de petróleo muestra un consumo medio de 312,351 unidades, mientras que el diésel, con un promedio de 869,783 unidades, tiene el rango más amplio y la desviación estándar más alta, lo que sugiere una mayor fluctuación en su consumo.
> - Los valores mínimos y máximos para cada tipo de combustible reflejan un rango amplio de consumo, con el diésel mostrando la mayor discrepancia entre los valores extremos. Estos resultados indican que, aunque hay una tendencia general en el consumo de cada tipo de combustible, hay una variabilidad considerable que podría estar influenciada por diversos factores económicos y políticos a lo largo del tiempo.

**Conjunto de Datos de Importación -->**

In [49]:
dataframes[1].describe()

,gasoline_regular,gasoline_superior,liquefied_gas,diesel
count,281.000000,2.810000e+02,2.810000e+02,2.810000e+02
mean,401433.017451,4.853736e+05,4.091257e+05,8.815024e+05
std,234159.202453,1.539939e+05,1.973531e+05,2.856735e+05
min,81015.300000,1.702925e+05,1.005615e+05,2.297647e+05
25%,205624.080000,3.668863e+05,2.252070e+05,6.869238e+05
50%,320566.690000,4.704840e+05,4.100011e+05,8.429935e+05
75%,548714.530000,5.958704e+05,5.656279e+05,1.095743e+06
max,987872.050000,1.227174e+06,1.077123e+06,1.595699e+06


**Observaciones 💡 -->**

El análisis descriptivo de las importaciones de productos derivados del petróleo en Guatemala muestra patrones interesantes en el consumo de diferentes tipos de combustibles.
> - La gasolina regular tiene una importación promedio de aproximadamente 401,433 unidades, con una desviación estándar considerable que sugiere variabilidad en el uso. Por otro lado, la gasolina superior, con un promedio de 485,374 unidades, presenta un rango de importación que va desde un mínimo de 170,292.5 hasta un máximo de 1,227,173.53, indicando que este combustible es utilizado en mayor medida en momentos específicos. El gas licuado de petróleo muestra una importación media de 409,126 unidades y una amplia desviación estándar, lo que también indica fluctuaciones significativas en su uso. En el caso del diésel, el promedio es de 881,502 unidades, con un rango considerable que refleja alta importación, alcanzando un máximo de aproximadamente 1,595,698.56.
> - Se cree que aunque hay un consumo promedio notable de todos los combustibles, la variabilidad en los datos puede estar influenciada por factores económicos, estacionales y políticos, lo que requiere un análisis más profundo para comprender las tendencias en el consumo de combustibles en el país.

### **(3) Análisis de Data Faltante**

In [50]:
# Checking null values of each columns
dataframes[0].isnull().sum()

gasoline_regular     0
gasoline_superior    0
liquefied_gas        0
diesel               0
dtype: int64

In [51]:
# Checking null values of each columns
dataframes[1].isnull().sum()

gasoline_regular     0
gasoline_superior    0
liquefied_gas        0
diesel               0
dtype: int64

**Observaciones 💡 -->**

Analisis aqui...